In [1]:
import numpy as np
import pandas as pd
import matplotlib.backends.backend_pdf
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from itertools import combinations


import scanpy as sc
import anndata as ad
import tangram as tg
import matplotlib.pyplot as plt
import seaborn as sns
import copy

np.random.seed(0)

In [2]:
project_location = "./"


In [4]:
# Load data

In [3]:
df_st = pd.read_csv(project_location + "data/merfish/merfishSpatial.csv", sep=" ")
coords = df_st['coord']
df_st.drop(columns=['coord'], inplace=True)
cellTypeCount =16
typeCountDf=  df_st.iloc[:,-cellTypeCount:]
# df_st = df_st.iloc[: , :-cellTypeCount]
df_st.head()

,Ace2,Adora2a,Aldh1l1,Amigo2,Ano3,Aqp4,Ar,Arhgap36,Avpr1a,Avpr2,...,ODImmature1,ODImmature2,Microglia,ODMature2,ODMature1,Endothelial3,ODMature3,ODMature4,Endothelial2,Ependymal
0,3.993327,20.596571,7.531108,13.294050,5.552498,9.445471,9.026888,23.262853,1.559171,0.000000,...,1,0,0,1,2,1,0,0,0,0
1,0.000000,0.000000,10.676513,0.606719,0.000000,43.836144,0.000000,8.300263,0.606719,1.437811,...,0,0,0,4,0,0,0,0,0,0
2,0.000000,0.000000,3.378244,1.219116,1.219116,7.910641,0.000000,1.529895,1.102756,0.000000,...,0,0,0,2,1,0,0,0,0,0
3,0.000000,0.000000,1.465245,0.732607,0.000000,2.323503,0.795436,0.732607,0.732607,0.000000,...,0,0,0,2,1,0,0,0,0,0
4,6.311896,7.574326,0.600405,0.600405,0.600405,0.600405,0.000000,0.600405,0.000000,0.000000,...,0,0,1,1,0,1,0,0,0,0


In [4]:
reference = pd.read_csv(project_location + "data/merfish/markerGene_for_merfish_data.csv")

reference.drop(columns=['p_value'], inplace=True)
# reference.drop(range(148,168))
reference.drop(reference.index[reference.cell_type=='EpendymalInhibitory'].tolist(), inplace=True)
reference['cell_type'] = reference['cell_type'].astype(str).str.replace(" ", "")
reference.head()

,cell_type,marker_gene
0,Astrocyte,Aldh1l1
1,Astrocyte,Aqp4
2,Astrocyte,Cxcl14
3,Astrocyte,Mlc1
4,Astrocyte,Ttyh2


In [5]:
genes = df_st.columns[:155]
markers = reference.groupby('cell_type').agg(list).marker_gene
celltypes = reference.cell_type.unique().tolist()
expressions = df_st.drop(columns=celltypes)
expressions = MinMaxScaler().fit_transform(expressions)
counts_st = pd.DataFrame(expressions, columns=genes)
cellcount = pd.DataFrame(np.sum(df_st[celltypes], axis=1), columns=['cellcount'])

## Single-cell data

In [6]:
df_sc = pd.read_csv(project_location + "data/merfish/merfishVisium.csv")
df_sc.head()

,Cell_ID,Animal_ID,Animal_sex,Behavior,Bregma,Centroid_X,Centroid_Y,Cell_class,Neuron_cluster_ID,Ace2,...,Penk,Scg2,Sln,Sst,Tac1,Tac2,Th,Trh,Ucn3,Vgf
0,1f9a8c19-b089-43d1-b609-7e791dc2c70f,1,Female,Naive,-0.24,-3749.176078,-3749.458442,Astrocyte,NaN,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.273327,0.0,0.003225,0.000000
1,b13e98f4-5c2b-4e96-985e-3e93aedc7221,1,Female,Naive,-0.24,-3746.226210,-3742.259347,Inhibitory,I-1,0.0,...,0.097542,0.0,0.000000,0.0,0.000000,0.016809,0.293094,0.0,0.006602,0.018258
2,d06cb29e-10ee-4bbc-b74e-90237999ef4b,1,Female,Naive,-0.24,-3742.897643,-3790.648737,Inhibitory,I-13,0.0,...,0.087302,0.0,0.063460,0.0,0.072791,0.000000,0.289555,0.0,0.000000,0.018325
3,12e2a165-57c7-4f37-96dd-23f6574af4ba,1,Female,Naive,-0.24,-3738.423005,-3773.259265,Inhibitory,I-19,0.0,...,0.098912,0.0,0.000000,0.0,0.048410,0.000000,0.247843,0.0,0.000000,0.008280
4,14a0f396-b13d-4d45-89a3-86c2047bf3f9,1,Female,Naive,-0.24,-3736.407620,-3895.590306,Pericytes,NaN,0.0,...,0.047054,0.0,0.178222,0.0,0.000000,0.000000,0.294404,0.0,0.000000,0.000000


In [7]:
counts_sc = df_sc[genes]

annotation= df_sc["Cell_class"]
frequency={}
for item in annotation:
    if item in frequency:
        frequency[item]+=1
    else:
        frequency[item]=1

frequency


{'Astrocyte': 745,
 'Inhibitory': 2219,
 'Pericytes': 62,
 'Ambiguous': 869,
 'Endothelial1': 301,
 'Excitatory': 1219,
 'ODImmature1': 223,
 'ODImmature2': 7,
 'Microglia': 106,
 'ODMature2': 241,
 'ODMature1': 49,
 'Endothelial3': 117,
 'ODMature3': 6,
 'ODMature4': 10,
 'Endothelial2': 19,
 'Ependymal': 219}

In [9]:
print(genes)

Index(['Ace2', 'Adora2a', 'Aldh1l1', 'Amigo2', 'Ano3', 'Aqp4', 'Ar',
       'Arhgap36', 'Avpr1a', 'Avpr2',
       ...
       'Penk', 'Scg2', 'Sln', 'Sst', 'Tac1', 'Tac2', 'Th', 'Trh', 'Ucn3',
       'Vgf'],
      dtype='object', length=155)


In [10]:
def  singleCellDf(twoTypes):
    global genes
    
    firstDf= df_sc.loc[df_sc['Cell_class']==twoTypes[0]]
    secondDf=df_sc.loc[df_sc['Cell_class']==twoTypes[1]]
    firstDf = firstDf[genes]
    secondDf= secondDf[genes]
    return [firstDf.head(5),secondDf.head(5)]
   
    
   

In [13]:
def mergedRow(firstDf,secondDf):
    concatDf = pd.concat([firstDf,secondDf])
    spot= concatDf.sum(axis=0)
    return spot    

    
    

In [16]:
def permutationOnaspot(firstDf,secondDf,twoType,spatialSpot):
    arr=[0,1,2,3,4]
    global genes
    combData=  pd.DataFrame(columns=genes)

    for i in range(1,5,1):
            

        permList =list(combinations(arr,i))
        permList = [list(elem) for elem in permList]
        tempData=  pd.DataFrame(columns=genes)





        for m in permList:
            for n in permList:
                exRows=firstDf.iloc[m,:]
                inRows= secondDf.iloc[n,:]
                exRows = pd.concat([exRows,inRows])
            
                spot= exRows.sum(axis=0)    
                tempData = tempData.append(spot,ignore_index=True)
                
        tempData[twoType[0]]= i
        tempData[twoType[1]] =i
        combData=pd.concat([combData,tempData])

    combData= combData.append(spatialSpot,ignore_index=True)

    stringVal=[]
    for i in range(len(combData)):
        res= str(i)+"x"+ str(i)
        stringVal.append(res)
    combData.insert(0,"coord",stringVal)
    combData["coord"]=combData.coord.astype(str)


    combData.to_csv("./data/twoType/spatialData/"+ twoType[0]+twoType[1]+ "Spatial.csv",index= False)



In [21]:
dataCellTypes= ["Astrocyte","Inhibitory","Pericytes","Ambiguous","Endothelial1","Excitatory","ODImmature1","ODImmature2","Microglia","ODMature2",
"ODMature1","Endothelial3","Ependymal","Endothelial2","ODMature3","ODMature4"]
twoTypeList =list(combinations(dataCellTypes,2))
twoTypeList = [list(elem) for elem in twoTypeList]
c=0
for t in twoTypeList:
    c+=1
    print(c)
    twoTypeSingleCell= df_sc[df_sc["Cell_class"].isin(t)]
    # resultSingleCell = twoTypeSingleCell.copy()
    resultSingleCell= pd.DataFrame(np.repeat(twoTypeSingleCell.to_numpy(), 126, axis=0), columns=twoTypeSingleCell.columns)
    # for i in range(125):
    #     resultSingleCell = pd.concat([resultSingleCell,twoTypeSingleCell])
    fileName = "./data/twoType/singleCell/"+t[0]+ t[1]+"Single.csv"
    resultSingleCell.to_csv(fileName,index= False)
    # firstTypeDf,secondTypeDf= singleCellDf(t)
    # superSpot = mergedRow(firstTypeDf,secondTypeDf)
    # permutationOnaspot(firstTypeDf,secondTypeDf,t,superSpot)
    
    
    

    


# Particular two cell type

In [9]:
excitatoryDf= df_sc.loc[df_sc['Cell_class']=="Excitatory"]
excitatoryDf =excitatoryDf[genes]
InhibitoryDf=df_sc.loc[df_sc['Cell_class']=="Inhibitory"]
InhibitoryDf=InhibitoryDf[genes]

options=["Excitatory","Inhibitory"]
rsltSingleCell= df_sc[df_sc["Cell_class"].isin(options)]


rsltSingleCell.to_csv("./data/FaultSingleCell.csv",index= False)

In [10]:
excitatoryTempDf= excitatoryDf.head(5)
InhibitoryTempDf=InhibitoryDf.head(5)
tempDf= pd.concat([excitatoryTempDf,InhibitoryTempDf])
tempDf.head()
tempDf.to_csv("./data/combinationSingleCell.csv",index=False)
print(tempDf.shape)

(10, 155)


In [11]:
rsltSingleCell.head()

,Cell_ID,Animal_ID,Animal_sex,Behavior,Bregma,Centroid_X,Centroid_Y,Cell_class,Neuron_cluster_ID,Ace2,...,Penk,Scg2,Sln,Sst,Tac1,Tac2,Th,Trh,Ucn3,Vgf
0,1f9a8c19-b089-43d1-b609-7e791dc2c70f,1,Female,Naive,-0.24,-3749.176078,-3749.458442,Astrocyte,NaN,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.273327,0.000000,0.003225,0.0
8,cfe60a3a-92a7-46d6-a2cf-687fd2af96fc,1,Female,Naive,-0.24,-3717.144006,-3760.757175,Excitatory,E-2,0.0,...,0.078634,0.0,0.0,0.0,0.050427,0.0,0.295925,0.000000,0.000000,0.0
9,4da51944-deb2-433c-843a-dfeb953186a3,1,Female,Naive,-0.24,-3716.115098,-3805.047855,Astrocyte,NaN,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.276880,0.013064,0.001580,0.0
12,59edae33-396d-49cc-8305-f2c477d83559,1,Female,Naive,-0.24,-3712.215305,-3717.469078,Astrocyte,NaN,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.287745,0.000000,0.000000,0.0
16,433723ce-eeb3-46a4-9ccc-d0d4f186ef2f,1,Female,Naive,-0.24,-3701.031349,-3749.273361,Astrocyte,NaN,0.0,...,0.000230,0.0,0.0,0.0,0.000000,0.0,0.254715,0.000214,0.000000,0.0


In [12]:
spotCount=200
randCount= np.random.randint(1,6,spotCount)
randCount

array([5, 1, 4, 4, 4, 2, 4, 3, 5, 1, 1, 5, 3, 2, 1, 2, 2, 1, 2, 5, 4, 1,
       4, 1, 3, 4, 1, 2, 4, 4, 4, 1, 2, 2, 2, 1, 3, 5, 4, 4, 3, 5, 3, 1,
       1, 5, 1, 5, 2, 5, 2, 3, 3, 1, 2, 2, 2, 2, 4, 4, 3, 4, 1, 4, 5, 2,
       3, 5, 4, 5, 5, 5, 4, 5, 5, 5, 1, 5, 4, 3, 1, 2, 2, 4, 1, 1, 2, 3,
       5, 3, 1, 4, 3, 3, 1, 2, 1, 3, 3, 4, 3, 4, 3, 2, 3, 4, 4, 4, 3, 4,
       5, 2, 3, 4, 2, 3, 2, 5, 3, 4, 1, 4, 3, 4, 1, 1, 1, 4, 3, 4, 1, 5,
       1, 1, 3, 4, 3, 4, 1, 1, 1, 4, 1, 3, 3, 1, 5, 4, 5, 1, 5, 4, 4, 5,
       2, 4, 1, 1, 1, 2, 1, 5, 2, 4, 2, 1, 1, 5, 4, 4, 2, 1, 1, 2, 3, 1,
       4, 2, 2, 5, 1, 1, 4, 3, 4, 3, 5, 4, 4, 1, 4, 1, 5, 3, 4, 5, 1, 3,
       4, 4])

In [13]:

count=0
spatialData= pd.DataFrame(columns=genes)

# exRows=excitatoryDf[count:(count+randCount[0])]
# inRows= InhibitoryDf[count:(count+randCount[0])]
# exRows = pd.concat([exRows,inRows])
# spot= exRows.sum(axis=0)    
# print(spot)


for i in range(spotCount):
    exRows=excitatoryDf[count:(count+randCount[i])]
    inRows= InhibitoryDf[count:(count+randCount[i])]
    exRows = pd.concat([exRows,inRows])
    
    spot= exRows.sum(axis=0)    
    spatialData = spatialData.append(spot,ignore_index=True)
    
    count+=randCount[i]
spatialData["Excitatory"]= randCount
spatialData["Inhibitory"] =randCount
spatialData.head()



stringVal=[]
for i in range(spotCount):
    res= str(i)+"x"+ str(i)
    stringVal.append(res)
    # print(spatialData.iloc[i]["coord"])
    # # print(res)
    # spatialData.iloc[i]["coord"]= " hello"
spatialData.insert(0, 'coord', stringVal)
spatialData["coord"]=spatialData.coord.astype(str)

spatialData.to_csv("./data/FaultSpatial.csv",index= False)
# print(spatialData.iloc[0]["coord"])

In [18]:
spatialData.head()

,coord,Ace2,Adora2a,Aldh1l1,Amigo2,Ano3,Aqp4,Ar,Arhgap36,Avpr1a,...,Sln,Sst,Tac1,Tac2,Th,Trh,Ucn3,Vgf,Excitatory,Inhibitory
0,0x0,0.360787,0.516422,17.601368,76.883393,2.953295,27.230260,22.420963,45.710406,2.156298,...,0.757329,0.0,0.378962,0.166447,2.591885,0.762383,0.046929,0.094774,5,5
1,1x1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.616830,0.000000,0.000000,...,0.548994,0.0,0.079511,0.000000,1.252596,0.000000,0.000000,0.010705,1,1
2,2x2,1.737130,2.260192,7.648333,106.473509,3.803570,7.554986,23.138114,51.391699,0.886948,...,0.385367,0.0,0.178470,0.213244,2.046420,1.658193,0.000000,0.474441,4,4
3,3x3,1.191190,1.257806,15.552129,104.832846,4.367850,21.683888,18.747099,43.980443,0.000000,...,0.708458,0.0,0.292812,1.288845,1.890644,0.862516,0.002071,0.046446,4,4
4,4x4,0.569134,0.700748,7.056597,86.148585,2.723574,3.137041,23.412328,40.100053,0.892213,...,0.915413,0.0,0.194141,0.138809,2.239634,1.006273,0.000000,0.464461,4,4


permutation on a spot


In [14]:
arr=[0,1,2,3,4]
combData=  pd.DataFrame(columns=genes)

for i in range(1,5,1):
        

    permList =list(combinations(arr,i))
    permList = [list(elem) for elem in permList]
    tempData=  pd.DataFrame(columns=genes)





    for m in permList:
        for n in permList:
            exRows=excitatoryDf.iloc[m,:]
            inRows= InhibitoryDf.iloc[n,:]
            exRows = pd.concat([exRows,inRows])
        
            spot= exRows.sum(axis=0)    
            tempData = tempData.append(spot,ignore_index=True)
            
    tempData["Excitatory"]= i
    tempData["Astrocyte"] =i
    combData=pd.concat([combData,tempData])

combData= combData.append(spatialData.iloc[0,:])
combData.pop("coord")

stringVal=[]
for i in range(len(combData)):
    res= str(i)+"x"+ str(i)
    stringVal.append(res)
combData.insert(0,"coord",stringVal)
combData["coord"]=combData.coord.astype(str)


combData.to_csv("./data/combinationSpatial.csv",index= False)



In [15]:
print(combData)

      coord      Ace2   Adora2a    Aldh1l1     Amigo2      Ano3        Aqp4  \
0       0x0  0.000000  0.000000  18.998619  10.176542  0.000000   39.728649   
1       1x1  0.000000  0.000000  18.199946   6.721826  0.000000   12.999096   
2       2x2  0.000000  0.000000  14.083426   9.282154  0.000000   17.923436   
3       3x3  0.000000  0.000000  26.027134   6.721826  0.000000   37.185634   
4       4x4  0.889688  0.000000  14.233511   6.721826  0.889688   63.165266   
..      ...       ...       ...        ...        ...       ...         ...   
21  246x246  1.250474  0.516422  68.819129  48.095350  3.212110  136.452264   
22  247x247  1.250474  0.516422  80.762837  45.535023  3.212110  155.714461   
23  248x248  1.250474  0.516422  76.646317  48.095350  3.212110  160.638801   
24  249x249  1.250474  0.516422  75.847644  44.640635  3.212110  133.909248   
0   250x250  1.250474  0.516422  94.846263  54.817176  3.212110  173.637897   

           Ar   Arhgap36    Avpr1a  ...      Sln  S

In [16]:
excitatoryDf.iloc[list(permList[0]),:]

,Ace2,Adora2a,Aldh1l1,Amigo2,Ano3,Aqp4,Ar,Arhgap36,Avpr1a,Avpr2,...,Penk,Scg2,Sln,Sst,Tac1,Tac2,Th,Trh,Ucn3,Vgf
8,0.0,0.000000,0.000000,6.721826,0.000000,0.000000,10.643941,1.120313,0.0,0.560163,...,0.078634,0.000000,0.00000,0.0,0.050427,0.0,0.295925,0.0,0.000000,0.00000
27,0.0,0.516422,2.582036,9.811850,0.000000,0.516422,3.614876,16.526694,0.0,0.000000,...,0.063624,0.000000,0.00000,0.0,0.011300,0.0,0.348714,0.0,0.000000,0.00000
29,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.279716,0.0,0.000000,...,0.062620,0.298614,0.00000,0.0,0.045796,0.0,0.237687,0.0,0.002602,0.00000
34,0.0,0.000000,0.000000,0.879274,0.879274,1.758608,5.275813,0.000000,0.0,0.000000,...,0.062713,0.000000,0.69387,0.0,0.150239,0.0,0.259476,0.0,0.000000,0.04991
